# Model Evaluation and Hyperparameter Tuning
In this notebook, I'll do the following for this week's assignment:
- Train multiple ML models
- Evaluate them using accuracy, precision, recall, and F1-score
- Use GridSearchCV and RandomizedSearchCV for hyperparameter tuning
- Select the best performing model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## Load and Prepare Dataset
I'm using the Breast Cancer dataset from `sklearn.datasets`.

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train and Evaluate Multiple Models

In [3]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    }
results_df = pd.DataFrame(results).T
results_df

D:\anaconda_new\envs\mindDigit\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Accuracy,Precision,Recall,F1 Score
Logistic Regression,0.956140,0.945946,0.985915,0.965517
Random Forest,0.964912,0.958904,0.985915,0.972222
Support Vector Machine,0.947368,0.922078,1.000000,0.959459
K-Nearest Neighbors,0.956140,0.934211,1.000000,0.965986


## Hyperparameter Tuning
Now I'll tune:
- **Random Forest** using `GridSearchCV`
- **SVM** using `RandomizedSearchCV`

In [4]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='f1', n_jobs=-1)
grid_rf.fit(X_train, y_train)
print("Best RF Params:", grid_rf.best_params_)
print("Best RF F1 Score (CV):", grid_rf.best_score_)

Best RF Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best RF F1 Score (CV): 0.9739085224112447


In [5]:
from scipy.stats import uniform
param_dist_svm = {
    'C': uniform(0.1, 10),
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear']
}
random_search_svm = RandomizedSearchCV(SVC(), param_distributions=param_dist_svm, 
                                       n_iter=20, cv=5, scoring='f1', random_state=42, n_jobs=-1)
random_search_svm.fit(X_train, y_train)
print("Best SVM Params:", random_search_svm.best_params_)
print("Best SVM F1 Score (CV):", random_search_svm.best_score_)

Best SVM Params: {'C': 3.1461376917337067, 'gamma': 'scale', 'kernel': 'linear'}
Best SVM F1 Score (CV): 0.9672532829401763


## Evaluate Tuned Models

In [6]:
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

best_svm = random_search_svm.best_estimator_
y_pred_svm = best_svm.predict(X_test)

final_results = {
    "Tuned Random Forest": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "Precision": precision_score(y_test, y_pred_rf),
        "Recall": recall_score(y_test, y_pred_rf),
        "F1 Score": f1_score(y_test, y_pred_rf)
    },
    "Tuned SVM": {
        "Accuracy": accuracy_score(y_test, y_pred_svm),
        "Precision": precision_score(y_test, y_pred_svm),
        "Recall": recall_score(y_test, y_pred_svm),
        "F1 Score": f1_score(y_test, y_pred_svm)
    }
}
final_results_df = pd.DataFrame(final_results).T
final_results_df

,Accuracy,Precision,Recall,F1 Score
Tuned Random Forest,0.964912,0.958904,0.985915,0.972222
Tuned SVM,0.956140,0.945946,0.985915,0.965517


## Conclusion

* **Random Forest** (both default and tuned) consistently delivers the **best F1 Score (0.972)**, which is crucial when balancing precision and recall.
* **SVM has perfect recall**, but its **lower precision** makes it less reliable if false positives matter.
* **Tuning SVM did not outperform the tuned Random Forest.**

### **Best Model Recommendation:**

> **Tuned Random Forest Classifier**
> Excellent overall metrics and consistent high performance across all categories.
